### Import des librairies
---

In [1]:

import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
import xgboost as xgb
import joblib

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_squared_error, r2_score

import matplotlib.pyplot as plt
import seaborn as sns

### Chargement des données
---

In [2]:
path = 'data/get_around_pricing_project.csv'
df = pd.read_csv(path, encoding='utf-8')
df.head()

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


In [3]:
df = df.drop(columns=['Unnamed: 0'])

### Modéle Linéaire Regression Baseline
---

In [4]:
# === 1. Séparation des données ===
target = "rental_price_per_day"
X = df.drop(columns=target)
y = df[target]

# === 2. Split du jeu de données ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === 3. Pipeline de prétraitement ===
numerical_columns = ['mileage', 'engine_power']
categorical_columns = ['model_key', 'fuel', 'paint_color', 'car_type',
                        'private_parking_available', 'has_gps', 'has_air_conditioning',
                        'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires']

numerical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("standardization", StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(drop="first"))
])

feature_encoder = ColumnTransformer(transformers=[
    ("num", numerical_pipeline, numerical_columns),
    ("cat", categorical_pipeline, categorical_columns)
])

# Transformation des données
X_train = feature_encoder.fit_transform(X_train)
X_test = feature_encoder.transform(X_test)

# === 4. Entraînement des modèles de régression === 
lin_reg = LinearRegression() 
lin_reg.fit(X_train, y_train)

# === 5. Prédictions ===
y_train_pred = lin_reg.predict(X_train)
y_test_pred = lin_reg.predict(X_test)

# === 6. Évaluation des performances ===
print("=== Score R2 ===")
print(f"R2 Score (Train): {r2_score(y_train, y_train_pred):.4f}")
print(f"R2 Score (Test) : {r2_score(y_test, y_test_pred):.4f}\n")

print("=== RMSE ===")
rmse = mean_squared_error(y_test, y_test_pred)
print(f"RMSE : {rmse:.2f}")

# Prix moyen sur tout le dataset
mean_price = df["rental_price_per_day"].mean()
print(f"Prix moyen : {mean_price:.2f} €")

from sklearn.metrics import median_absolute_error

medae = median_absolute_error(y_test, y_test_pred)
print(f"Median AE : {medae:.2f} €")




=== Score R2 ===
R2 Score (Train): 0.7140
R2 Score (Test) : 0.6937

=== RMSE ===
RMSE : 322.59
Prix moyen : 121.21 €
Median AE : 8.19 €


### Modéle RandomForestRegressor
---
#### RandomForestRegressor prédit un prix en construisant plusieurs arbres de décision indépendants et en moyennant leurs prédictions, ce qui rend la prédiction plus stable et robuste.

In [5]:

target = "rental_price_per_day"
X = df.drop(columns=target)
y = df[target]

# Split du jeu de données ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline de prétraitement ===
numerical_columns = ['mileage', 'engine_power']
categorical_columns = ['model_key', 'fuel', 'paint_color', 'car_type',
                        'private_parking_available', 'has_gps', 'has_air_conditioning',
                        'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires']

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("standardization", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_columns),
        ("cat", categorical_transformer, categorical_columns)
    ])

# Création du pipeline complet
# n_estimators=100 → assez d’arbres pour stabiliser le modèle.
# max_depth=9 → limite la complexité des arbres pour mieux généraliser sur le test.
# random_state=42 → assure que les résultats sont toujours identiques.
model = RandomForestRegressor(n_estimators=100, max_depth=12, random_state=42) #100 arbres dans la foret - chaque arbre à max 9 niveaux de decisions - 
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", model) 
])

# Entraînement
pipeline.fit(X_train, y_train)

# Prédiction 
y_pred = pipeline.predict(X_test)

y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

# Évaluation des performances

print("=== Score R2 ===")
print(f"R2 Score (Train): {r2_score(y_train, y_train_pred):.4f}")
print(f"R2 Score (Test) : {r2_score(y_test, y_test_pred):.4f}\n")

print("=== RMSE ===")
rmse = mean_squared_error(y_test, y_test_pred)
print(f"RMSE : {rmse:.2f}")

# Prix moyen sur tout le dataset
mean_price = df["rental_price_per_day"].mean()
print(f"Prix moyen : {mean_price:.2f} €")

medae = median_absolute_error(y_test, y_test_pred)
print(f"Median AE : {medae:.2f} €")

print("\nLe modèle prédit très bien la tendance générale (R² test = 0,72).\nLe RMSE est élevé car il est sensible aux voitures très chères ou très bon marché,\nmais la Median Absolute Error de 7,1€ montre que pour la majorité des véhicules,\nnos prédictions sont très proches du prix réel.")


=== Score R2 ===
R2 Score (Train): 0.9273
R2 Score (Test) : 0.7386

=== RMSE ===
RMSE : 275.32
Prix moyen : 121.21 €
Median AE : 7.18 €

Le modèle prédit très bien la tendance générale (R² test = 0,72).
Le RMSE est élevé car il est sensible aux voitures très chères ou très bon marché,
mais la Median Absolute Error de 7,1€ montre que pour la majorité des véhicules,
nos prédictions sont très proches du prix réel.


### Modéle XGBoost
---
#### XGBoost (eXtreme Gradient Boosting) est un algorithme de Gradient Boosting basé sur des arbres de décision, optimisé pour la vitesse et la performance, où chaque nouvel arbre corrige les erreurs des arbres précédents.

In [6]:
# Target et features
target = "rental_price_per_day"
X = df.drop(columns=target)
y = df[target]

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Colonnes numériques et catégorielles
numerical_columns = ['mileage', 'engine_power']
categorical_columns = ['model_key', 'fuel', 'paint_color', 'car_type',
                       'private_parking_available', 'has_gps', 'has_air_conditioning',
                       'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires']

# Prétraitement numérique
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Prétraitement catégoriel
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# Combine transformations
preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, numerical_columns),
    ("cat", categorical_transformer, categorical_columns)
])

# Modèle XGBoost
xgb_model = xgb.XGBRegressor(
    n_estimators=200,      # nombre d'arbres
    max_depth=9,           # profondeur maximale
    learning_rate=0.1,     # taux d'apprentissage : Contrôle combien chaque arbre corrige l’erreur du précédent. Plus petit = apprentissage plus lent et stable.
    subsample=0.8,         # échantillonnage pour régularisation : fraction des échantillons utilisés pour chaque arbre.Introduit un peu d’aléatoire pour réduire l’overfitting.
)

# Pipeline complet
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", xgb_model)
])

# Entraînement
pipeline.fit(X_train, y_train)

# Prédiction
y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

# Évaluation
print("=== Score R2 ===")
print(f"R2 Score (Train): {r2_score(y_train, y_train_pred):.4f}")
print(f"R2 Score (Test) : {r2_score(y_test, y_test_pred):.4f}\n")

print("=== RMSE ===")
rmse = mean_squared_error(y_test, y_test_pred)
print(f"RMSE : {rmse:.2f}")

mean_price = df["rental_price_per_day"].mean()
print(f"Prix moyen : {mean_price:.2f} €")

medae = median_absolute_error(y_test, y_test_pred)
print(f"Median AE : {medae:.2f} €")

print("\nLe modèle XGBoost améliore le R2 test avec 0.74, cependant il overfite avec un train à 0.98 (écart à 0.16)")



=== Score R2 ===
R2 Score (Train): 0.9886
R2 Score (Test) : 0.7406

=== RMSE ===
RMSE : 273.23
Prix moyen : 121.21 €
Median AE : 6.44 €

Le modèle XGBoost améliore le R2 test avec 0.74, cependant il overfite avec un train à 0.98 (écart à 0.16)


### Modéle XGBoost + GridSearch
---
#### GridSearchCV est un outil qui permet de chercher les meilleures combinaisons de paramètres pour un modèle, afin d’optimiser sa performance. Après avoir entraîné le GridSearch, je récupère best_estimator_, qui contient le pipeline complet optimisé, et je le sauvegarde immédiatement avec joblib.dump. Cela me permet de le recharger plus tard pour prédire sur de nouvelles données sans refaire le prétraitement ni réentraîner le modèle

In [7]:
# Target et features
target = "rental_price_per_day"
X = df.drop(columns=target)
y = df[target]

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Colonnes numériques et catégorielles
numerical_columns = ['mileage', 'engine_power']
categorical_columns = ['model_key', 'fuel', 'paint_color', 'car_type',
                       'private_parking_available', 'has_gps', 'has_air_conditioning',
                       'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires']

# Prétraitement numérique
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Prétraitement catégoriel
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# Combine transformations
preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, numerical_columns),
    ("cat", categorical_transformer, categorical_columns)
])

# Modèle XGBoost
xgb_model = xgb.XGBRegressor(
    random_state=42,
    tree_method="hist"
)

# Pipeline complet
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", xgb_model)
])

# Paramètres pour GridSearch (réduits pour éviter surcharge)
param_grid = {
    "regressor__max_depth": [3, 4],
    "regressor__learning_rate": [0.05, 0.1],
    "regressor__n_estimators": [100, 200],
    "regressor__subsample": [0.8],
}

# GridSearch avec 3-fold CV pour limiter le temps
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,
    scoring="r2",
    n_jobs=-1,
    verbose=2
)

# Entraînement GridSearch
grid_search.fit(X_train, y_train)

# Meilleurs paramètres et score CV
print("Meilleurs paramètres : ", grid_search.best_params_)
print("Meilleur R2 CV : ", grid_search.best_score_)

# Sauvegarde du pipeline complet
# On récupère le pipeline optimisé (prétraitement + XGBoost avec les meilleurs hyperparamètres)
best_model = grid_search.best_estimator_

# Sauvegarde au format .pkl
joblib.dump(best_model, "modele_xgb_getaround.pkl")
print("✅ Modèle sauvegardé sous : modele_xgb_getaround.pkl")


# Prédictions avec le meilleur modèle
y_train_pred = grid_search.best_estimator_.predict(X_train)
y_test_pred = grid_search.best_estimator_.predict(X_test)

# Évaluation
print("=== Score R2 ===")
print(f"R2 Score (Train): {r2_score(y_train, y_train_pred):.4f}")
print(f"R2 Score (Test) : {r2_score(y_test, y_test_pred):.4f}\n")

print("=== RMSE ===")
rmse = mean_squared_error(y_test, y_test_pred)
print(f"RMSE : {rmse:.2f}")

mean_price = df["rental_price_per_day"].mean()
print(f"Prix moyen : {mean_price:.2f} €")

medae = median_absolute_error(y_test, y_test_pred)
print(f"Median AE : {medae:.2f} €")

print("\nAprès optimisation via GridSearch, le modèle présente une meilleure performance avec un R² test de 0.75,\ntout en réduisant l’overfitting avec le R² train de 0.82 (écart de 0.07).\nLe modèle est ainsi plus robuste et stable.")


Fitting 3 folds for each of 8 candidates, totalling 24 fits
Meilleurs paramètres :  {'regressor__learning_rate': 0.05, 'regressor__max_depth': 4, 'regressor__n_estimators': 200, 'regressor__subsample': 0.8}
Meilleur R2 CV :  0.7517865300178528
✅ Modèle sauvegardé sous : modele_xgb_getaround.pkl
=== Score R2 ===
R2 Score (Train): 0.8234
R2 Score (Test) : 0.7470

=== RMSE ===
RMSE : 266.42
Prix moyen : 121.21 €
Median AE : 6.84 €

Après optimisation via GridSearch, le modèle présente une meilleure performance avec un R² test de 0.75,
tout en réduisant l’overfitting avec le R² train de 0.82 (écart de 0.07).
Le modèle est ainsi plus robuste et stable.


### Enregistrement du modèle et des métriques dans MLflow
---
### Après la recherche d’hyperparamètres avec GridSearch, j’ai récupéré grid_search.best_estimator_, qui contient le pipeline complet (prétraitement + XGBoost optimisé). Je l’ai sauvegardé avec joblib.dump. Ensuite, dans mon application, je recharge ce fichier avec joblib.load, ce qui me permet de prédire directement sur de nouvelles données sans refaire tout le preprocessing. »

In [8]:
# # Démarre une session MLflow
# with mlflow.start_run():
#     # Enregistre le pipeline complet
#     mlflow.sklearn.log_model(pipeline, "random_forest_model")
    
#     # Log des métriques calculées
#     mlflow.log_metric("R2_train", r2_score(y_train, y_train_pred))
#     mlflow.log_metric("R2_test", r2_score(y_test, y_test_pred))
#     mlflow.log_metric("RMSE", mean_squared_error(y_test, y_test_pred))  # RMSE = sqrt(MSE)
#     mlflow.log_metric("MedianAE", median_absolute_error(y_test, y_test_pred))
    
#     print("Modèle et métriques enregistrés dans MLflow")
